In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ls "/content/drive/My Drive/Deep Learning/data/"

sample_submission.csv  submission_gru.csv  test.csv  train.csv


In [0]:
#Importing modules
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import keras.backend as K
from keras import optimizers
from keras.engine.topology import Layer
from keras.models import Input, Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D,MaxPooling1D, Flatten
from keras.layers import GlobalMaxPool1D,SpatialDropout1D,Activation, Lambda
from sklearn.model_selection import train_test_split
from keras.models import Model,load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

#Evaluation Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support


In [0]:
train = pd.read_csv('/content/drive/My Drive/Deep Learning/offense/taskb/taskb_combined_mstr_data.csv',encoding='utf-8')
submit = pd.read_csv('/content/drive/My Drive/Deep Learning/offense/taskb/testset-taskb.tsv',delimiter='\t',encoding='utf-8')
submit_template = pd.read_csv('/content/drive/My Drive/Deep Learning/offense/taskb/taskb_prediction_submission_template.csv', header = 0)


In [0]:
#performing blank(null) check
null_check=train.isnull().sum()
print(null_check)

In [0]:
#Split input to train and test
x_train, x_test, y_train, y_test = train_test_split(train, train["tag"], test_size = 0.10, random_state = 42)

In [0]:
#Storing the Toxic content column for further processing
list_sentences_train = x_train["comment"]
list_sentences_test = x_test["comment"]
list_sentences_submit = submit["tweet"]

In [0]:
##Cleaning the text
import re,string
from string import digits

def remove_non_ascii(text):
    return ''.join(i for i in text if ord(i)<128)
  
list_sentences_train = list_sentences_train.apply(lambda x: re.sub(' +', ' ',x))
list_sentences_test = list_sentences_test.apply(lambda x: re.sub(' +', ' ',x))
list_sentences_submit = list_sentences_submit.apply(lambda x: re.sub(' +', ' ',x))

list_sentences_train = list_sentences_train.apply(lambda x: x.lower())
list_sentences_test = list_sentences_test.apply(lambda x: x.lower())
list_sentences_submit = list_sentences_submit.apply(lambda x: x.lower())

list_sentences_train = list_sentences_train.apply(lambda x: x.replace('user',''))
list_sentences_test = list_sentences_test.apply(lambda x: x.replace('user',''))
list_sentences_submit = list_sentences_submit.apply(lambda x: x.replace('user',''))

list_sentences_train = list_sentences_train.apply(lambda x: x.replace('url',''))
list_sentences_test = list_sentences_test.apply(lambda x: x.replace('url',''))
list_sentences_submit = list_sentences_submit.apply(lambda x: x.replace('url',''))

list_sentences_train = list_sentences_train.apply(lambda x: re.sub('http\S+\s*', '', x))
list_sentences_test = list_sentences_test.apply(lambda x: re.sub('http\S+\s*', '', x))
list_sentences_submit = list_sentences_submit.apply(lambda x: re.sub('http\S+\s*', '', x))
                                                    
list_sentences_train = list_sentences_train.apply(lambda x: re.sub('@\S+', '', x))
list_sentences_test = list_sentences_test.apply(lambda x: re.sub('@\S+', '', x))
list_sentences_submit = list_sentences_submit.apply(lambda x: re.sub('@\S+', '', x))  

list_sentences_train = list_sentences_train.apply(lambda x: re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', x))
list_sentences_test = list_sentences_test.apply(lambda x: re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', x))
list_sentences_submit = list_sentences_submit.apply(lambda x: re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', x))                                                       
                                                    
list_sentences_train = list_sentences_train.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
list_sentences_test = list_sentences_test.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
list_sentences_submit = list_sentences_submit.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))

list_sentences_train = list_sentences_train.apply(remove_non_ascii)
list_sentences_test = list_sentences_test.apply(remove_non_ascii)
list_sentences_submit = list_sentences_submit.apply(remove_non_ascii)

list_sentences_train = list_sentences_train.apply(lambda x: x.strip())
list_sentences_test = list_sentences_test.apply(lambda x: x.strip())
list_sentences_submit = list_sentences_submit.apply(lambda x: x.strip())

list_sentences_train = list_sentences_train.apply(lambda x: re.sub('[^a-zA-Z0-9\n\.]', ' ', x))
list_sentences_test = list_sentences_test.apply(lambda x: re.sub('[^a-zA-Z0-9\n\.]', ' ', x))
list_sentences_submit = list_sentences_submit.apply(lambda x: re.sub('[^a-zA-Z0-9\n\.]', ' ', x))

list_sentences_train = list_sentences_train.apply(lambda x: re.sub('\n', ' ', x))
list_sentences_test = list_sentences_test.apply(lambda x: re.sub('\n', ' ', x))
list_sentences_submit = list_sentences_submit.apply(lambda x: re.sub('\n', ' ', x))

list_sentences_train = list_sentences_train.apply(lambda x: re.sub(' +', ' ',x))
list_sentences_test = list_sentences_test.apply(lambda x: re.sub(' +', ' ',x))
list_sentences_submit = list_sentences_submit.apply(lambda x: re.sub(' +', ' ',x))


In [0]:
def channel_normalization(x):
    # type: (Layer) -> Layer
    """ Normalize a layer to the maximum activation
    This keeps a layers values between zero and one.
    It helps with relu's unbounded activation
    Args:
        x: The layer to normalize
    Returns:
        A maximal normalized layer
    """
    max_values = K.max(K.abs(x), 2, keepdims=True) + 1e-5
    out = x / max_values
    return out


def residual_block(x, s, i, activation, nb_filters, kernel_size, dropout_rate=0):
    # type: (Layer, int, int, str, int, int, float) -> Tuple[Layer, Layer]
    """Defines the residual block TCN
    Args:
        x: The previous layer in the model
        s: The stack index i.e. which stack in the overall TCN
        i: The dilation power of 2 we are using for this residual block
        activation: The name of the type of activation to use
        nb_filters: The number of convolutional filters to use in this block
        kernel_size: The size of the convolutional kernel
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
    Returns:
        A tuple where the first element is the residual model layer, and the second
        is the skip connection.
    """
    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding='causal',
                  name='dilated_conv_%d_tanh_s%d' % (i, s))(x)
    if activation == 'norm_relu':
        x = Activation('relu')(conv)
        x = Lambda(channel_normalization)(x)
    else:
        x = Activation(activation)(conv)

    x = SpatialDropout1D(dropout_rate, name='spatial_dropout1d_%d_s%d_%f' % (i, s, dropout_rate))(x)

    # 1x1 conv.
    x = Convolution1D(nb_filters, 1, padding='causal')(x)
    res_x = keras.layers.add([original_x, x])
    return res_x, x


def process_dilations(dilations):
    def is_power_of_two(num):
        return num != 0 and ((num & (num - 1)) == 0)

    if all([is_power_of_two(i) for i in dilations]):
        return dilations

    else:
        new_dilations = [2 ** i for i in dilations]
        print(f'Updated dilations from {dilations} to {new_dilations} because of backwards compatibility.')
        return new_dilations


def TCN(input_layer,
        embedding_param,
        nb_filters=64,
        kernel_size=2,
        nb_stacks=1,
        dilations=None,
        activation='norm_relu',
        use_skip_connections=True,
        dropout_rate=0.0,
        return_sequences=True):
    """Creates a TCN layer.
    Args:
        input_layer: A tensor of shape (batch_size, timesteps, input_dim).
        nb_filters: The number of filters to use in the convolutional layers.
        kernel_size: The size of the kernel to use in each convolutional layer.
        dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
        nb_stacks : The number of stacks of residual blocks to use.
        activation: The activations to use (norm_relu, wavenet, relu...).
        use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
        return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
    Returns:
        A TCN layer.
    """

    ###
    print('TCN: embedding_param=', embedding_param)
    print('TCN: nb_filters=', nb_filters)
    print('TCN: kernel_size=', kernel_size)
    print('TCN: nb_stacks=', nb_stacks)
    print('TCN: dilations=', dilations)
    print('TCN: activation=', activation)
    print('TCN: use_skip_connections=', use_skip_connections)
    print('TCN: dropout_rate=', dropout_rate)
    print('TCN: return_sequences=', return_sequences)
    ###
   
    if dilations is None:
        dilations = [1, 2, 4, 8, 16, 32]
    embed_size = 240
    x = Embedding(embedding_param, embed_size)(input_layer)

    ###x = input_layer
    x = Convolution1D(nb_filters, 1, padding='causal', name='initial_conv')(x) 
    skip_connections = []
    for s in range(nb_stacks):
        for i in dilations:
            x, skip_out = residual_block(x, s, i, activation, nb_filters, kernel_size, dropout_rate)
            skip_connections.append(skip_out)
    if use_skip_connections:
        x = keras.layers.add(skip_connections)
    
    x = Activation('relu')(x)

    if not return_sequences:
        output_slice_index = -1
        x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
    return x


def compiled_tcn(num_feat,  # type: int
                 num_classes,  # type: int
                 nb_filters,  # type: int
                 kernel_size,  # type: int
                 embedding_param,
                 dilations,  # type: List[int]
                 nb_stacks,  # type: int
                 max_len,  # type: int
                 activation='norm_relu',  # type: str
                 use_skip_connections=True,  # type: bool
                 return_sequences=True,
                 regression=False,  # type: bool
                 dropout_rate=0.05  # type: float
                 ):
    # type: (...) -> keras.Model
    """Creates a compiled TCN model for a given task (i.e. regression or classification).
    Args:
        num_feat: A tensor of shape (batch_size, timesteps, input_dim).
        num_classes: The size of the final dense layer, how many classes we are predicting.
        nb_filters: The number of filters to use in the convolutional layers.
        kernel_size: The size of the kernel to use in each convolutional layer.
        dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
        nb_stacks : The number of stacks of residual blocks to use.
        max_len: The maximum sequence length, use None if the sequence length is dynamic.
        activation: The activations to use.
        use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
        return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
        regression: Whether the output should be continuous or discrete.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
    Returns:
        A compiled keras TCN.
    """
    ##process tcn dilations
    dilations = process_dilations(dilations)

    input_layer = Input(name='input_layer', shape=(max_len, ))   

    x = TCN(input_layer,embedding_param, nb_filters, kernel_size, nb_stacks, dilations, activation,
            use_skip_connections, dropout_rate, return_sequences)

    print('compiled tcn: x.shape=', x.shape)
    print('compiled tcn: max_len=', max_len)
    print('compiled tcn: num_feat=', num_feat)
    print('compiled tcn: num_classes=', num_classes)
    print('compiled tcn: embedding_param=', embedding_param)
    print('compiled tcn: nb_filters=', nb_filters)
    print('compiled tcn: kernel_size=', kernel_size)
    print('compiled tcn: nb_stacks=', nb_stacks)
    print('compiled tcn: dilations=', dilations)
    print('compiled tcn: activation=', activation)
    print('compiled tcn: use_skip_connections=', use_skip_connections)
    print('compiled tcn: dropout_rate=', dropout_rate)
    print('compiled tcn: return_sequences=', return_sequences)

    if not regression:
        ###Fully Connected
        x = GlobalMaxPool1D()(x)
        x = Dense(50,activation="relu")(x)
        x = Dropout(0.2)(x)
      
        ###
        # classification
        x = Dense(num_classes)(x)
        x = Activation('sigmoid', name='output_sigmoid')(x)
        output_layer = x
        print(f'model.x = {input_layer.shape}')
        print(f'model.y = {output_layer.shape}')
        model = Model(input_layer, output_layer)

        adam = optimizers.Adam(lr=1e-4, clipnorm=1.)
        model.compile(adam, loss='binary_crossentropy', metrics=['accuracy'])
        print('Adam with norm clipping.')

    model_name = 'D-TCN_C{}_B{}_L{}'.format(2, nb_stacks, '-'.join(map(str, dilations)))
    print(f'Model name = {model_name}.')
    return model

In [0]:
#Wrapper which generates TCN, fits the model and outputs the prediction file
def run_task():

    max_features = 20000
    tokenizer = Tokenizer(num_words=max_features)

    tokenizer.fit_on_texts(list(list_sentences_train))

    list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
    list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
    list_tokenized_submit = tokenizer.texts_to_sequences(list_sentences_submit)

    maxlen = 500
    X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
    X_sub = pad_sequences(list_tokenized_submit, maxlen=maxlen)

    #Compiling a TCN model with following set parameters
    model = compiled_tcn(num_feat=20000,
                                num_classes=6, 
                                nb_filters=500,
                                kernel_size=3,
                                embedding_param=len(tokenizer.word_index)+1, 
                                dilations=[2 ** i for i in range(10)],
                                nb_stacks=1,
                                max_len=500,    
                                activation='norm_relu',
                                use_skip_connections=True,
                                return_sequences=True)



    model.summary()
    ##Pausing Early Stopping temporarily for observing convergence 
    #early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=2, verbose=1)
    #callbacks_list = [early_stopping]

    filepath="/content/drive/My Drive/Deep Learning/tcn_chk_points/weights-improvement-{epoch:02d}-{val_acc:.6f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]    
    
    #Fitting the model
    hist=model.fit(X_t, y_train,batch_size=32, epochs=3,validation_data=(X_te, y_test),validation_split=0.1,shuffle=True,verbose=1) ##model.fit(x_train, y_train.squeeze().argmax(axis=1), epochs=15,


    #Prediction      
    y_submit = model.predict(X_sub,batch_size=32,verbose=1)
    y_submit[np.isnan(y_submit)]=0
    sample_submission = submit_template
    sample_submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_submit
    sample_submission.to_csv('/content/drive/My Drive/Deep Learning/data/submission_tcn.csv', index=False)      

###
if __name__ == '__main__':
    run_task()

TCN: embedding_param= 197608
TCN: nb_filters= 500
TCN: kernel_size= 3
TCN: nb_stacks= 1
TCN: dilations= [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
TCN: activation= norm_relu
TCN: use_skip_connections= True
TCN: dropout_rate= 0.05
TCN: return_sequences= True
compiled tcn: x.shape= (?, 500, 500)
compiled tcn: max_len= 500
compiled tcn: num_feat= 20000
compiled tcn: num_classes= 6
compiled tcn: embedding_param= 197608
compiled tcn: nb_filters= 500
compiled tcn: kernel_size= 3
compiled tcn: nb_stacks= 1
compiled tcn: dilations= [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
compiled tcn: activation= norm_relu
compiled tcn: use_skip_connections= True
compiled tcn: dropout_rate= 0.05
compiled tcn: return_sequences= True
model.x = (?, 500)
model.y = (?, 6)
Adam with norm clipping.
Model name = D-TCN_C2_B1_L1-2-4-8-16-32-64-128-256-512.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Conne